In [ ]:
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
%matplotlib qt

file = '../data/STO/haadf.hspy'

s = hs.load(file)
image = s.data	
dx = s.axes_manager[1].scale *10 # convert nm to Angstrom
model=ImageModelFitting(image, dx=dx,units='A')

In [ ]:
model.plot() # inspect the image

In [ ]:
model.find_peaks(min_distance=10,threshold_rel=0.05) # find atomic columns by local maximum

In [ ]:
cif_file = '../data/STO/SrTiO3_mp-5229_conventional_standard.cif' 
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'], reciprocal=False,region_index=0) # map the cif unitcell to the image and tile it according the local peak positions

In [5]:
model.view_3d() # view the 3D model using ase of Atoms object, with coordinates refined from the image

In [ ]:
model.select_atoms() # select the region where has atoms

In [6]:
model.plot_coordinates(10) # plot the coordinates of the selected region with element labels

In [ ]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=10, tol=1e-2, plot=False, step_size=0.01,verbose=False,num_epoch=1) # set the maxiter and tol and num_epoch to be larger for better fitting

In [ ]:
model.plot_fitting() # compare the fitting result with the original image

In [11]:
model.plot_scs(per_element=True) # plot the scattering cross section of the elements in the selected region, show each element separately

In [14]:
model.plot_scs_histogram() # plot the histogram of the scattering cross section of the selected region

In [ ]:
model.voronoi_integration() # integrate the scattering cross section of the selected region using voronoi tessellation

In [59]:
model.plot_scs_voronoi() # plot the scattering cross section of the selected region using voronoi tessellation